# signature retrain

In [1]:
import os
import numpy as np
import tensorflow as tf
import pandas as pd
from PIL import UnidentifiedImageError
from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dense, Lambda, Dropout, BatchNormalization, Activation, GlobalAveragePooling2D
from tensorflow.keras.regularizers import l2
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard
from sklearn.model_selection import train_test_split

In [2]:
# Charger les paires existantes
df = pd.read_csv('signature_pairs.csv', sep=";")

def load_and_preprocess_image(img_path, target_size):
    try:
        img = load_img(img_path, target_size=target_size, color_mode='grayscale')
        img = img_to_array(img) / 255.0
        return img
    except UnidentifiedImageError:
        print(f"Erreur: Impossible de charger l'image {img_path}.")
        return None

# Charger et prétraiter les images
img_size = (128, 128)
pairs = []
labels = []

for index, row in df.iterrows():
    img1 = load_and_preprocess_image(row['image_1'], img_size)
    img2 = load_and_preprocess_image(row['image_2'], img_size)
    
    if img1 is not None and img2 is not None:
        pairs.append([img1, img2])
        labels.append(row['label'])

pairs = np.array(pairs)
labels = np.array(labels)

# Diviser les données en ensembles d'entraînement et de validation
pairs_train, pairs_val, labels_train, labels_val = train_test_split(pairs, labels, test_size=0.2, random_state=42)

In [3]:
def contrastive_loss(y_true, y_pred, margin=1.0):
    y_true = tf.cast(y_true, tf.float32)
    loss_similar = (1 - y_true) * 0.5 * K.square(y_pred)
    loss_dissimilar = y_true * 0.5 * K.square(K.maximum(margin - y_pred, 0))
    return K.mean(loss_similar + loss_dissimilar)

def create_base_network(input_shape):
    input = Input(shape=input_shape)
    
    # First Convolutional Block
    x = Conv2D(64, (3, 3), padding='same', kernel_regularizer=l2(1e-4))(input)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(64, (3, 3), padding='same', kernel_regularizer=l2(1e-4))(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    
    # Second Convolutional Block
    x = Conv2D(128, (3, 3), padding='same', kernel_regularizer=l2(1e-4))(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(128, (3, 3), padding='same', kernel_regularizer=l2(1e-4))(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    
    # Third Convolutional Block
    x = Conv2D(256, (3, 3), padding='same', kernel_regularizer=l2(1e-4))(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(256, (3, 3), padding='same', kernel_regularizer=l2(1e-4))(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    
    # Global Average Pooling
    x = GlobalAveragePooling2D()(x)
    
    # Fully Connected Layers
    x = Dense(512, activation='relu', kernel_regularizer=l2(1e-4))(x)
    x = Dropout(0.5)(x)
    x = Dense(128, activation='relu', kernel_regularizer=l2(1e-4))(x)
    
    return Model(input, x)

input_shape = (128, 128, 1)
base_network = create_base_network(input_shape)

input_a = Input(shape=input_shape)
input_b = Input(shape=input_shape)

processed_a = base_network(input_a)
processed_b = base_network(input_b)

def euclidean_distance(vects):
    x, y = vects
    sum_square = K.sum(K.square(x - y), axis=1, keepdims=True)
    return K.sqrt(K.maximum(sum_square, K.epsilon()))

distance = Lambda(euclidean_distance)([processed_a, processed_b])
output = Dense(1, activation='sigmoid')(distance)
model = Model([input_a, input_b], output)

2024-09-21 19:55:32.526842: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Pro
2024-09-21 19:55:32.526928: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-09-21 19:55:32.526954: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-09-21 19:55:32.527046: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-09-21 19:55:32.527085: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [4]:
# Charger les poids sauvegardés
model.load_weights('./models/best_model_V5_continue.keras')

# Compilation du modèle
optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=0.0001)
model.compile(loss=contrastive_loss, optimizer=optimizer, metrics=['accuracy'])

# Augmentation des données
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('models/best_model_V6.keras', save_best_only=True, monitor='val_accuracy', mode='max')
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.00001, verbose=1)
tensorboard = TensorBoard(log_dir='./logs', histogram_freq=1, write_graph=True, write_images=True)

# Entraînement du modèle
history = model.fit(
    datagen.flow([pairs_train[:, 0], pairs_train[:, 1]], labels_train, batch_size=32),
    steps_per_epoch=len(pairs_train) // 32,
    epochs=100,
    validation_data=([pairs_val[:, 0], pairs_val[:, 1]], labels_val),
    callbacks=[early_stopping, model_checkpoint, reduce_lr, tensorboard])

# Évaluation du modèle
val_loss, val_accuracy = model.evaluate([pairs_val[:, 0], pairs_val[:, 1]], labels_val)
print(f"Validation Loss: {val_loss:.4f}")
print(f"Validation Accuracy: {val_accuracy:.4f}")

Epoch 1/100


2024-09-21 19:55:45.681108: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


881/881 [==============================] - 368s 409ms/step - loss: 0.0380 - accuracy: 0.9238 - val_loss: 0.0305 - val_accuracy: 0.9457 - lr: 1.0000e-04
Epoch 2/100
881/881 [==============================] - 348s 395ms/step - loss: 0.0339 - accuracy: 0.9248 - val_loss: 0.0263 - val_accuracy: 0.9336 - lr: 1.0000e-04
Epoch 3/100
881/881 [==============================] - 348s 395ms/step - loss: 0.0318 - accuracy: 0.9247 - val_loss: 0.0240 - val_accuracy: 0.9427 - lr: 1.0000e-04
Epoch 4/100
881/881 [==============================] - 348s 395ms/step - loss: 0.0314 - accuracy: 0.9243 - val_loss: 0.0216 - val_accuracy: 0.9437 - lr: 1.0000e-04
Epoch 5/100
881/881 [==============================] - 1916s 2s/step - loss: 0.0316 - accuracy: 0.9234 - val_loss: 0.0233 - val_accuracy: 0.9359 - lr: 1.0000e-04
Epoch 6/100
881/881 [==============================] - 349s 396ms/step - loss: 0.0297 - accuracy: 0.9253 - val_loss: 0.0216 - val_accuracy: 0.9501 - lr: 1.0000e-04
Epoch 7/100
881/881 [=========

TypeError: Object of type float32 is not JSON serializable